In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
import subprocess

import napari
from magicgui import magicgui

from multiplex_pipeline.core_definition.viewer_utils import (
    display_saved_rois,
    redo_bbox_layer,
    redo_cores_layer,
    save_rois_from_viewer,
)
from multiplex_pipeline.utils.im_utils import get_org_im_shape, prepare_rgb_image
from multiplex_pipeline.core_definition.roi_utils import (
    get_refined_rectangles,
    get_visual_rectangles,
    prepare_poly_df_for_saving,
    xywh_to_corners,
)
from multiplex_pipeline.utils.file_utils import (
    change_to_wsl_path
)
from multiplex_pipeline.utils.config_loaders import (
    load_analysis_settings,
    load_workstation_config
)

## Read in config

In [2]:
# load pipeline configuration
config = load_workstation_config(r'R:\Wayne\BLCA\pipeline_settings.yaml')
config

{'model_path': '/mnt/d/data_analysis/2024_bladder/sam2',
 'sam_env': '/home/kasia/miniforge3/envs/sam2-env/bin/python'}

In [3]:
# load analysis configuration
settings_path = r'R:/Wayne/BLCA/BLCA-2_Analysis/analysis_settings_BLCA2.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-2/BLCA-2_Final',
 'analysis_name': 'BLCA-2_Analysis',
 'local_analysis_dir': 'R:/Wayne/BLCA/BLCA-2_Analysis',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/BLCA-2_Analysis/logs'),
 'detection_image': 'BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/BLCA-2_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/BLCA-2_Analysis/temp'),
 'cores_dir_output': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/BLCA-2_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  '

In [4]:
# simplify variable names
im_level = settings['core_detection']['im_level']
save_path = settings['core_info_file_path']

## Read in data

In [5]:
# read in image data

# define the image path
image_path = os.path.join(settings['image_dir'], settings['detection_image'])

# get the original image shape
org_im_shape = get_org_im_shape(image_path)

# prepare the image for display
im_rgb = prepare_rgb_image(image_path, req_level = im_level)
im = im_rgb[:, :, 0]
im_rgb.shape

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


(1015, 755, 3)

## Create a viewer with options

In [6]:
viewer = napari.Viewer()
viewer.add_image(im)

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im.shape[1],im.shape[0]], frame = 0)
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=2, name = 'frame')

# add a layer for the saved rois
display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)


#########################################################################################
# add diplay saved button
@magicgui(auto_call=False, call_button="Display Saved Cores", result_widget=False)
def display_saved_rois_button(viewer: napari.Viewer):
    display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(display_saved_rois_button, area='left')

#########################################################################################
# add a save button
@magicgui(auto_call=False, call_button="Save Cores")
def save_button(viewer: napari.Viewer):
    save_rois_from_viewer(viewer, org_im_shape = org_im_shape, req_level = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')

## Get suggestions for core detection from the SAM2 model

Execute the cell below to get suggestions from SAM model about the cores position. 

Sometimes there is no immediate feedback of how SAM2 progresses. In this situation it's expected to wait 1-2 min for the outcome.

In [7]:
# Linux path to the image
im_wsl = change_to_wsl_path(image_path)

# temporary output path for masks
out_win = os.path.join(os.getcwd(), 'masks.pkl')
out_wsl = change_to_wsl_path(out_win)

# Combine the command
command = [
    "wsl",
    config['sam_env'],
    "-m", "multiplex_pipeline.core_selection.suggest_cores",
    im_wsl,
    str(im_level),
    config['model_path'],
    out_wsl
]

# Execute the script
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Pipe the standard output
    stderr=subprocess.PIPE,  # Pipe the standard error
    text=True                # Capture output as text
)

# Display output in real time
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")  # Print each line as it becomes available

# Print any remaining errors
errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

# load the masks
masks = pkl.load(open(out_win, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# remove the masks file
os.remove(out_win)

# refine masks
rect_list = get_refined_rectangles(masks, im = im_rgb[:,:,0],
                                   frame = settings['core_detection']['frame'],
                                   min_area = settings['core_detection']['min_area'],
                                   max_area = settings['core_detection']['max_area'],
                                   min_iou = settings['core_detection']['min_iou'],
                                   min_stability = settings['core_detection']['min_st'],
                                   min_int = settings['core_detection']['min_int']
                                   )
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = im_level)
rect_list = get_visual_rectangles(df, im_level)
poly_list = [(x/(2**im_level)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list())
redo_bbox_layer(viewer,rect_list,df['core_name'].tolist())

ERROR: /home/kasia/miniforge3/envs/sam2-env/bin/python: Error while finding module specification for 'multiplex_pipeline.core_selection.suggest_cores' (ModuleNotFoundError: No module named 'multiplex_pipeline.core_selection')



FileNotFoundError: [Errno 2] No such file or directory: 'd:\\multiplex_pipeline\\notebooks\\masks.pkl'